In [1]:
import tropycal.tracks as tracks
import datetime as dt

/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/plot/plot.py:16: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn("Warning: Cartopy is not installed in your python environment. Plotting functions will not work.")
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tracks/plot.py:24: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn("Warning: Cartopy is not installed in your python environment. Plotting functions will not work.")
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tornado/dataset.py:21: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn(warn_message)
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tornado/plot.py:21: UserWarning: Warning: Cartopy is not installed in your python environm

In [2]:
#read north atlantic datasets
hurdat_atl = tracks.TrackDataset(basin='north_atlantic',source='hurdat',include_btk=False)

--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (4.72 seconds)


In [3]:
storm = hurdat_atl.get_storm(('michael',2018))
print(storm.to_dataframe().head())

                 date  extra_obs special type   lat   lon  vmax  mslp  \
0 2018-10-06 18:00:00          0           LO  17.8 -86.6    25  1006   
1 2018-10-07 00:00:00          0           LO  18.1 -86.9    25  1004   
2 2018-10-07 06:00:00          0           TD  18.4 -86.8    30  1004   
3 2018-10-07 12:00:00          0           TS  18.8 -86.4    35  1003   
4 2018-10-07 18:00:00          0           TS  19.1 -85.7    45   999   

        wmo_basin  
0  north_atlantic  
1  north_atlantic  
2  north_atlantic  
3  north_atlantic  
4  north_atlantic  


In [27]:
# storm.plot(return_ax=True)

# get_operational_forecasts()

#Method 1: Specify date closest to desired discussion
disco = storm.get_nhc_discussion(forecast=dt.datetime(2018,10,6,0))
print(disco['text'])

ZCZC MIATCDAT4 ALL
TTAA00 KNHC DDHHMM

Potential Tropical Cyclone Fourteen Discussion Number   1
NWS National Hurricane Center Miami FL       AL142018
400 PM CDT Sat Oct 06 2018

Satellite imagery and surface observations indicate that the
circulation of the low pressure area in the northwestern Caribbean
Sea is getting better defined, and that the associated convection
is becoming better organized.  While the system is currently not
well enough organized to call it a tropical depression, current
indications in the global models and the intensity guidance are that
the system will develop into a tropical cyclone within 24 h and
could bring tropical storm conditions to portions of western Cuba
and the northeastern Yucatan Peninsula.  Based on the need for
warnings and watches in these areas, advisories are being initiated
on Potential Tropical Cyclone Fourteen.

Although the intensity guidance is in excellent agreement that the
system should strengthen through the forecast period, the gl

In [26]:
import pandas as pd
pd.DataFrame(storm.get_operational_forecasts()['CARQ']['2018100600'])

,fhr,lat,lon,vmax,mslp,type,init
0,-24,14.9,-83.5,20,NaN,TD,2018-10-06
1,-18,15.3,-83.8,20,NaN,TD,2018-10-06
2,-12,15.7,-84.0,20,NaN,TD,2018-10-06
3,-6,16.0,-84.3,25,NaN,TD,2018-10-06
4,0,16.5,-84.7,25,1007.0,TD,2018-10-06


In [6]:
storm.plot_nhc_forecast(forecast=2)

NameError: name 'ccrs' is not defined